In [ ]:
%%markdown
# INFO-F-422 -  Statistical Foundations of Machine Learning 

### Gian Marco Paldino - [gian.marco.paldino@ulb.be](mailto:gian.marco.paldino@ulb.be)
### Cédric Simar - [cedric.simar@ulb.be](mailto:cedric.simar@ulb.be)

## TP 5 - Ensembles of models and feature selection

#### April 2023

#### Materials originally developed by *Yann-Aël Le Borgne, Fabrizio Carcillo and Gianluca Bontempi*

In [ ]:
%%markdown
## Overview

Feature selection and ensembles of models are two techniques which can be used to improve the accuracy of predictions.

Feature selection aims at reducing the dimensionality of the problem, and is useful when input variables contain redundant or irrelevant (noisy) information. Benefits are twofold: it decreases the training time by simplifying the problem, and it decreases the complexity of the predictive model. This in turn usually improves the prediction accuracy, since high-dimensionality makes predictive models more prone to overfitting, and estimates of parameters more variant.
s
There are three main approaches to feature selection:
- **Filter methods:** 
These methods rely solely on the data and their intrinsic properties, without considering the impact of the selected features on the learning algorithm performance. For this reason, they are often used as preprocessing techniques.

- **Wrapper methods:** 
These methods assess subsets of variables according to their usefulness to a given predictor. The feature selection is performed using an evaluation function that includes the predictive performance of the considered learning algorithm as a selection criterion.

- **Embedded methods:** 
These methods are specific to given learning machines, and usually built-in in the learning procedure (e.g. random forest, regularization based techniques).

Ensembles of models consist in building several predictive models using resampled subsets of the original training set. The method works particularly well for predictive models with high variance (for example, decision trees or neural networks). The average prediction of the resulting models usually strongly decreases the variance component of the error, and as a consequence improves the prediction accuracy.

In this session, we will illustrate both techniques using the IMDB 5000 dataset, which contains 27 variables describing 5043 movies. The variables contain information about the director, actors, number of Facebook likes for each actor, duration, genre, language, country, etc... We will use them to predict the movie success (through the IMDB score). The dataset together with a description of the variables is at https://www.kaggle.com/deepmatrix/imdb-5000-movie-dataset.

The dataset is on the github of the course, in `5_EnsemblesFeatureSelection/movie_metadata.csv`

In [ ]:
%%markdown
## Preliminaries

### Supervised learning

The process of supervised learning involves the presence of an entity (the learner, also called prediction model), whose goal is to learn the mapping between inputs and outputs in a given problem.

A supervised learning problem can be formulated as follows:

\[
 y = m(\mathbf{x})
\]

where:
- \(y\) represents the output variable (also called target)
- \(\mathbf{x}\) represents the vector of inputs (also called features).
- \(m\) is the (unknown) mapping between input and outputs.

In the majority of the supervised learning problems, the mapping \(m\) between input and outputs is unknown and needs to be estimated on the basis of the available input/output observation pairs \((\mathbf{x}_i,y_i)\).

## Classification vs regression

Both classification and regression are sub-fields of *supervised learning*. In the two cases, we have predictive variables \(\mathbf{x}\) and a target variable \(y\). 
The main difference between the two type of problems is the type of the target variable:

- In classification, \(y\) is a discrete variable; i.e \(y \in \{C_1,\cdots,C_k\}\)
- In regression, \(y\) is a continuous variable; i.e \(y \in \mathbb{R}\)

In this practical, unlike the previous ones, we will tackle our problem as a regression problem, with the IMDB score being the continuous target variable to predict.

In [ ]:
%%markdown
## Data overview and preprocessing

In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Load dataset
data = pd.read_csv("movie_metadata.csv")
np.random.seed(2)
data = data.sample(1000) # Random subset of 1000 movies

print(data.shape)
data.head(2)

In [ ]:
data.describe(include='all')

In [ ]:
%%markdown
We see there is a mix of categorical and numerical variables, and some missing values. In order to simplify the analysis, let us remove the categorical variables, and replace the NA values with the mean values of the variables.

In [ ]:
# Identify factor (categorical) variables
factor_cols = data.select_dtypes(include=['object']).columns
factor_cols

In [ ]:
# Remove categorical variables
data_preprocessed = data.drop(columns=factor_cols)

# Replace NaN with mean
data_preprocessed = data_preprocessed.apply(lambda x: x.fillna(x.mean()))

data_preprocessed.describe()

In [ ]:
%%markdown
### Input and output variables

The output variable (Y) is the `imdb_score`, and all other variables (X) are considered as inputs.

In [ ]:
Y = data_preprocessed['imdb_score'].values
X = data_preprocessed.drop(columns=['imdb_score'])

N = X.shape[0]
n = X.shape[1]

import matplotlib.pyplot as plt

plt.hist(Y, bins=20)
plt.title("Distribution of imdb_score")
plt.show()

print("Mean of Y:", np.mean(Y))
print("Variance of Y:", np.var(Y))

In [ ]:
%%markdown
### 1) Modelling with linear and decision tree models

#### Linear model

* Create a linear model for predicting the IMDB score on the basis of the other variables, and compute its empirical mean square error

In [ ]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(X, Y)
Y_hat = model.predict(X)

empirical_error = np.mean((Y_hat - Y)**2)
print("Empirical error=", round(empirical_error,4))

In [ ]:
%%markdown
* Which input variables are statistically correlated with the output?

In Python, we can check the coefficients of the linear model.

In [ ]:
coefs = pd.Series(model.coef_, index=X.columns)
coefs

In [ ]:
%%markdown
* Compute the validation error with a 10-fold cross-validation

In [ ]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=10, shuffle=True, random_state=3)
CV_err_lm_single_model = []

for train_index, test_index in kf.split(X):
    X_tr, X_ts = X.iloc[train_index], X.iloc[test_index]
    Y_tr, Y_ts = Y[train_index], Y[test_index]
    
    model = LinearRegression()
    model.fit(X_tr, Y_tr)
    Y_hat_ts = model.predict(X_ts)
    CV_err_lm_single_model.append(np.mean((Y_hat_ts - Y_ts)**2))

print("CV error=", round(np.mean(CV_err_lm_single_model),4), 
      "; std dev=", round(np.std(CV_err_lm_single_model),4))

In [ ]:
%%markdown
#### Decision tree

* Modify the previous code to compute the empirical error using a decision tree model. Use sklearn's `DecisionTreeRegressor`.

In [ ]:
from sklearn.tree import DecisionTreeRegressor

model = DecisionTreeRegressor(random_state=3)
model.fit(X, Y)
Y_hat = model.predict(X)

empirical_error = np.mean((Y_hat - Y)**2)
print("Empirical error=", round(empirical_error,4))

In [ ]:
%%markdown
* Plot the resulting tree is more complicated in Python due to size, but we can just visualize its structure or get the depth.

In [ ]:
model.get_depth(), model.get_n_leaves()

In [ ]:
%%markdown
* What is the 10-fold cross-validation error using a decision tree model?

In [ ]:
CV_err_rpart_single_model = []

for train_index, test_index in kf.split(X):
    X_tr, X_ts = X.iloc[train_index], X.iloc[test_index]
    Y_tr, Y_ts = Y[train_index], Y[test_index]
    
    model = DecisionTreeRegressor(random_state=3)
    model.fit(X_tr, Y_tr)
    Y_hat_ts = model.predict(X_ts)
    CV_err_rpart_single_model.append(np.mean((Y_hat_ts - Y_ts)**2))

print("CV error=", round(np.mean(CV_err_rpart_single_model),4), 
      "; std dev=", round(np.std(CV_err_rpart_single_model),4))

In [ ]:
%%markdown
## 2) Ensemble of models

Let us now create an ensemble of R=20 linear models to make predictions.

In [ ]:
R = 20
CV_err_lm_ensemble_model = []

for train_index, test_index in kf.split(X):
    X_tr, X_ts = X.iloc[train_index], X.iloc[test_index]
    Y_tr, Y_ts = Y[train_index], Y[test_index]
    
    Y_hat_ts_ensemble = np.zeros((X_ts.shape[0], R))
    for r in range(R):
        # Resample the training indices
        idx_tr_resample = np.random.choice(train_index, size=len(train_index), replace=True)
        X_tr_res = X.iloc[idx_tr_resample]
        Y_tr_res = Y[idx_tr_resample]
        
        model = LinearRegression()
        model.fit(X_tr_res, Y_tr_res)
        Y_hat_ts_ensemble[:, r] = model.predict(X_ts)
    
    Y_hat_ts = np.mean(Y_hat_ts_ensemble, axis=1)
    CV_err_lm_ensemble_model.append(np.mean((Y_hat_ts - Y_ts)**2))

print("CV error=", round(np.mean(CV_err_lm_ensemble_model),4), 
      "; std dev=", round(np.std(CV_err_lm_ensemble_model),4))

# Is the CV error lower?
print("Is ensemble error lower than single model?", 
      np.mean(CV_err_lm_ensemble_model) < np.mean(CV_err_lm_single_model))

In [ ]:
%%markdown
* Use a decision tree as the base model. Is the CV error lower?

In [ ]:
R = 20
CV_err_rpart_ensemble_model = []

for train_index, test_index in kf.split(X):
    X_tr, X_ts = X.iloc[train_index], X.iloc[test_index]
    Y_tr, Y_ts = Y[train_index], Y[test_index]
    
    Y_hat_ts_ensemble = np.zeros((X_ts.shape[0], R))
    for r in range(R):
        idx_tr_resample = np.random.choice(train_index, size=len(train_index), replace=True)
        X_tr_res = X.iloc[idx_tr_resample]
        Y_tr_res = Y[idx_tr_resample]
        
        model = DecisionTreeRegressor(random_state=r)
        model.fit(X_tr_res, Y_tr_res)
        Y_hat_ts_ensemble[:, r] = model.predict(X_ts)
    
    Y_hat_ts = np.mean(Y_hat_ts_ensemble, axis=1)
    CV_err_rpart_ensemble_model.append(np.mean((Y_hat_ts - Y_ts)**2))

print("CV error=", round(np.mean(CV_err_rpart_ensemble_model),4), 
      "; std dev=", round(np.std(CV_err_rpart_ensemble_model),4))

# Is ensemble error lower?
print("Is ensemble error lower than single tree model?", 
      np.mean(CV_err_rpart_ensemble_model) < np.mean(CV_err_rpart_single_model))

In [ ]:
%%markdown
## 3) Feature selection

### Filter methods

#### Correlation with the output

The following code performs feature selection by keeping the most correlated variables with the output. Compare the results for linear models and decision trees. What are the smallest CV errors, and how many features were needed?

In [ ]:
correlations = np.abs(X.corrwith(pd.Series(Y)))
ranking_corr_idx = correlations.sort_values(ascending=False).index

CV_err = np.zeros((n,10))

fold_id = 0
for train_index, test_index in kf.split(X):
    X_tr, X_ts = X.iloc[train_index], X.iloc[test_index]
    Y_tr, Y_ts = Y[train_index], Y[test_index]
    
    for nb_features in range(1, n+1):
        selected_features = ranking_corr_idx[:nb_features]
        model = LinearRegression()
        model.fit(X_tr[selected_features], Y_tr)
        Y_hat_ts = model.predict(X_ts[selected_features])
        CV_err[nb_features-1, fold_id] = np.mean((Y_hat_ts - Y_ts)**2)
    fold_id += 1

for i in range(n):
    print("#Features:", i+1, "; CV error=", round(np.mean(CV_err[i,:]),4),
          "; std dev=", round(np.std(CV_err[i,:]),4))

print("Correlation ranking:")
print(ranking_corr_idx.tolist())

In [ ]:
%%markdown
#### mRMR

We will implement a simple mRMR feature selection. mRMR uses mutual information. We will approximate mutual information via the correlation-based formula provided.

In [ ]:
def mutual_info_corr(X, Y):
    c = np.corrcoef(X, Y)[0,1]
    # Avoid invalid value if correlation == 1 or == -1
    if abs(c)==1:
        c = 0.999999
    return -0.5 * np.log(1 - c**2)

def compute_mi_vector(X_tr, Y_tr):
    mis = []
    for col in X_tr.columns:
        mi = mutual_info_corr(X_tr[col].values, Y_tr)
        mis.append(mi)
    return np.array(mis)

CV_err = np.zeros((n,10))

fold_id = 0
for train_index, test_index in kf.split(X):
    X_tr, X_ts = X.iloc[train_index], X.iloc[test_index]
    Y_tr, Y_ts = Y[train_index], Y[test_index]
    
    mutual_info_values = compute_mi_vector(X_tr, Y_tr)
    selected = []
    candidates = list(range(n))
    
    for j in range(n):
        redundancy_score = np.zeros(len(candidates))
        if len(selected)>0:
            # Compute pairwise mi between selected and candidates
            mi_sc = []
            for cidx in candidates:
                col_c = X_tr.iloc[:, cidx]
                mis_c = []
                for sidx in selected:
                    col_s = X_tr.iloc[:, sidx]
                    # Compute mutual info between col_s and col_c
                    cc = np.corrcoef(col_s, col_c)[0,1]
                    if abs(cc)==1:
                        cc=0.999999
                    mis_c.append(-0.5*np.log(1-cc**2))
                redundancy_score[candidates.index(cidx)] = np.mean(mis_c)
        mRMR_score = mutual_info_values[candidates] - redundancy_score
        best_idx = candidates[np.argmax(mRMR_score)]
        selected.append(best_idx)
        candidates.remove(best_idx)
    
    # selected is the ranking
    for nb_features in range(1, n+1):
        features_to_use = [X.columns[i] for i in selected[:nb_features]]
        model = LinearRegression()
        model.fit(X_tr[features_to_use], Y_tr)
        Y_hat_ts = model.predict(X_ts[features_to_use])
        CV_err[nb_features-1, fold_id] = np.mean((Y_hat_ts - Y_ts)**2)
    fold_id += 1

for i in range(n):
    print("#Features:", i+1, "; CV error=", round(np.mean(CV_err[i,:]),4),
          "; std dev=", round(np.std(CV_err[i,:]),4))
    
print("Selected features ranking (mRMR):")
print([X.columns[i] for i in selected])

In [ ]:
%%markdown
#### PCA

The following code performs features selection by first transforming the inputs using PCA, and then keeping the most relevant principal components in the model.

In [ ]:
from sklearn.decomposition import PCA

pca = PCA()
X_pca = pca.fit_transform(X)

CV_err = np.zeros((n,10))
fold_id = 0
for train_index, test_index in kf.split(X_pca):
    X_tr, X_ts = X_pca[train_index], X_pca[test_index]
    Y_tr, Y_ts = Y[train_index], Y[test_index]
    
    for nb_components in range(1, n+1):
        model = LinearRegression()
        model.fit(X_tr[:, :nb_components], Y_tr)
        Y_hat_ts = model.predict(X_ts[:, :nb_components])
        CV_err[nb_components-1, fold_id] = np.mean((Y_hat_ts - Y_ts)**2)
    fold_id += 1

for i in range(n):
    print("#Features:", i+1, "; CV error=", round(np.mean(CV_err[i,:]),4),
          "; std dev=", round(np.std(CV_err[i,:]),4))

In [ ]:
%%markdown
### Wrapper method: Forward selection

In [ ]:
# Forward selection
selected = []
for round_i in range(n):
    candidates = list(set(range(n)) - set(selected))
    CV_err_temp = []
    for c in candidates:
        features_to_include = selected + [c]
        fold_errors = []
        for train_index, test_index in kf.split(X):
            X_tr, X_ts = X.iloc[train_index, features_to_include], X.iloc[test_index, features_to_include]
            Y_tr, Y_ts = Y[train_index], Y[test_index]
            
            model = LinearRegression()
            model.fit(X_tr, Y_tr)
            Y_hat_ts = model.predict(X_ts)
            fold_errors.append(np.mean((Y_hat_ts - Y_ts)**2))
        CV_err_temp.append(np.mean(fold_errors))
    
    best_candidate = candidates[np.argmin(CV_err_temp)]
    selected.append(best_candidate)
    print("Round", round_i+1, "; Selected feature:", best_candidate,
          "; CV error=", round(min(CV_err_temp),4), 
          "; std dev=", round(np.std(fold_errors),4))

print("Selected features:", [X.columns[i] for i in selected])

In [ ]:
%%markdown
## Further preprocessing to add categorical variables

Categorical variables usually need to be transformed with 'one-hot-encoding' in order to be processed by a learning algorithm.

In the following, we add some categorical variables (color, language, country, content_rating) to the preprocessed dataset, and use them to improve prediction performance.

We can do the one-hot encoding using pandas `get_dummies`.

In [ ]:
factor_vars = factor_cols
data_factor = data[factor_vars]

variables_to_keep = ["color","language","country","content_rating"]

data_factor_onehot = pd.get_dummies(data_factor[variables_to_keep], prefix=variables_to_keep, dummy_na=False)

data_preprocessed_extended = pd.concat([data_preprocessed, data_factor_onehot], axis=1)
X = data_preprocessed_extended.drop(columns=['imdb_score'])
Y = data_preprocessed_extended['imdb_score'].values
N, n = X.shape

X.columns = [c.replace(" ", "_").replace("-", "_") for c in X.columns]

In [ ]:
%%markdown
## Using other predictive models

We can try other models like SVM (SVR), Neural Networks (MLPRegressor), K-Nearest Neighbors (KNeighborsRegressor) and see if performance improves.

In [ ]:
from sklearn.svm import SVR

CV_err_svm_single_model = []
for train_index, test_index in kf.split(X):
    X_tr, X_ts = X.iloc[train_index], X.iloc[test_index]
    Y_tr, Y_ts = Y[train_index], Y[test_index]
    
    model = SVR()
    model.fit(X_tr, Y_tr)
    Y_hat_ts = model.predict(X_ts)
    CV_err_svm_single_model.append(np.mean((Y_hat_ts - Y_ts)**2))

print("CV error=",round(np.mean(CV_err_svm_single_model),4),
      "; std dev=", round(np.std(CV_err_svm_single_model),4))

In [ ]:
# Feature selection with mRMR for SVM

n_variables = 10

CV_err_svm_single_model_fs = np.zeros((n_variables,10))
fold_id = 0

for train_index, test_index in kf.split(X):
    X_tr, X_ts = X.iloc[train_index], X.iloc[test_index]
    Y_tr, Y_ts = Y[train_index], Y[test_index]
    
    mutual_info_values = compute_mi_vector(X_tr, Y_tr)
    selected = []
    candidates = list(range(n))
    
    for j in range(n_variables):
        redundancy_score = np.zeros(len(candidates))
        if len(selected)>0:
            for ci, cidx in enumerate(candidates):
                col_c = X_tr.iloc[:, cidx].values
                mis_c = []
                for sidx in selected:
                    col_s = X_tr.iloc[:, sidx].values
                    cc = np.corrcoef(col_s, col_c)[0,1]
                    if abs(cc)==1:
                        cc=0.999999
                    mis_c.append(-0.5*np.log(1-cc**2))
                redundancy_score[ci] = np.mean(mis_c)
        mRMR_score = mutual_info_values[candidates] - redundancy_score
        best_idx = candidates[np.argmax(mRMR_score)]
        selected.append(best_idx)
        candidates.remove(best_idx)
        
    # Evaluate performance with subsets of selected features
    for nb_features in range(1, n_variables+1):
        feats = [X.columns[i] for i in selected[:nb_features]]
        model = SVR()
        model.fit(X_tr[feats], Y_tr)
        Y_hat_ts = model.predict(X_ts[feats])
        CV_err_svm_single_model_fs[nb_features-1, fold_id] = np.mean((Y_hat_ts - Y_ts)**2)
    fold_id+=1

for i in range(n_variables):
    print("#Features:", i+1, "; CV error=",round(np.mean(CV_err_svm_single_model_fs[i,:]),4),
          "; std dev=", round(np.std(CV_err_svm_single_model_fs[i,:]),4))

In [ ]:
from sklearn.neural_network import MLPRegressor

CV_err_nnet_single_model = []

for train_index, test_index in kf.split(X):
    X_tr, X_ts = X.iloc[train_index], X.iloc[test_index]
    Y_tr, Y_ts = Y[train_index], Y[test_index]
    
    # Rescale output to 0-1
    Y_tr_rescale = Y_tr/10.0
    model = MLPRegressor(hidden_layer_sizes=(10,), max_iter=5000, random_state=3)
    model.fit(X_tr, Y_tr_rescale)
    Y_hat_ts = model.predict(X_ts)*10.0
    CV_err_nnet_single_model.append(np.mean((Y_hat_ts - Y_ts)**2))

print("CV error=",round(np.mean(CV_err_nnet_single_model),4),
      "; std dev=", round(np.std(CV_err_nnet_single_model),4))

In [ ]:
# Feature selection for MLP using mRMR
CV_err_nnet_single_model_fs = np.zeros((n_variables,10))
fold_id=0
for train_index, test_index in kf.split(X):
    X_tr, X_ts = X.iloc[train_index], X.iloc[test_index]
    Y_tr, Y_ts = Y[train_index], Y[test_index]
    
    mutual_info_values = compute_mi_vector(X_tr, Y_tr)
    selected = []
    candidates = list(range(n))
    
    for j in range(n_variables):
        redundancy_score = np.zeros(len(candidates))
        if len(selected)>0:
            for ci, cidx in enumerate(candidates):
                col_c = X_tr.iloc[:, cidx].values
                mis_c = []
                for sidx in selected:
                    col_s = X_tr.iloc[:, sidx].values
                    cc = np.corrcoef(col_s, col_c)[0,1]
                    if abs(cc)==1:
                        cc=0.999999
                    mis_c.append(-0.5*np.log(1-cc**2))
                redundancy_score[ci] = np.mean(mis_c)
        mRMR_score = mutual_info_values[candidates] - redundancy_score
        best_idx = candidates[np.argmax(mRMR_score)]
        selected.append(best_idx)
        candidates.remove(best_idx)
        
    for nb_features in range(1, n_variables+1):
        feats = [X.columns[i] for i in selected[:nb_features]]
        Y_tr_rescale = Y_tr/10.0
        model = MLPRegressor(hidden_layer_sizes=(10,), max_iter=500, random_state=3)
        model.fit(X_tr[feats], Y_tr_rescale)
        Y_hat_ts = model.predict(X_ts[feats])*10.0
        CV_err_nnet_single_model_fs[nb_features-1, fold_id] = np.mean((Y_hat_ts - Y_ts)**2)
    fold_id+=1

for i in range(n_variables):
    print("#Features:", i+1, "; CV error=",round(np.mean(CV_err_nnet_single_model_fs[i,:]),4),
          "; std dev=", round(np.std(CV_err_nnet_single_model_fs[i,:]),4))

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

CV_err_lazy_single_model = []
for train_index, test_index in kf.split(X):
    X_tr, X_ts = X.iloc[train_index], X.iloc[test_index]
    Y_tr, Y_ts = Y[train_index], Y[test_index]
    
    model = KNeighborsRegressor(n_neighbors=5)
    model.fit(X_tr, Y_tr)
    Y_hat_ts = model.predict(X_ts)
    CV_err_lazy_single_model.append(np.mean((Y_hat_ts - Y_ts)**2))

print("CV error=",round(np.mean(CV_err_lazy_single_model),4),
      "; std dev=", round(np.std(CV_err_lazy_single_model),4))

In [ ]:
# Feature selection for KNN using mRMR
CV_err_lazy_single_model_fs = np.zeros((n_variables,10))
fold_id=0
for train_index, test_index in kf.split(X):
    X_tr, X_ts = X.iloc[train_index], X.iloc[test_index]
    Y_tr, Y_ts = Y[train_index], Y[test_index]
    
    mutual_info_values = compute_mi_vector(X_tr, Y_tr)
    selected = []
    candidates = list(range(n))
    
    for j in range(n_variables):
        redundancy_score = np.zeros(len(candidates))
        if len(selected)>0:
            for ci, cidx in enumerate(candidates):
                col_c = X_tr.iloc[:, cidx].values
                mis_c = []
                for sidx in selected:
                    col_s = X_tr.iloc[:, sidx].values
                    cc = np.corrcoef(col_s, col_c)[0,1]
                    if abs(cc)==1:
                        cc=0.999999
                    mis_c.append(-0.5*np.log(1-cc**2))
                redundancy_score[ci] = np.mean(mis_c)
        mRMR_score = mutual_info_values[candidates] - redundancy_score
        best_idx = candidates[np.argmax(mRMR_score)]
        selected.append(best_idx)
        candidates.remove(best_idx)
        
    for nb_features in range(1, n_variables+1):
        feats = [X.columns[i] for i in selected[:nb_features]]
        model = KNeighborsRegressor(n_neighbors=5)
        model.fit(X_tr[feats], Y_tr)
        Y_hat_ts = model.predict(X_ts[feats])
        CV_err_lazy_single_model_fs[nb_features-1, fold_id] = np.mean((Y_hat_ts - Y_ts)**2)
    fold_id+=1

for i in range(n_variables):
    print("#Features:", i+1, "; CV error=",round(np.mean(CV_err_lazy_single_model_fs[i,:]),4),
          "; std dev=", round(np.std(CV_err_lazy_single_model_fs[i,:]),4))